In [1]:
import re
import pandas as pd
import json


In [8]:
import os

def list_files(startpath):
    for root, dirs, files in os.walk(startpath):
        for file in files:
            yield os.path.join(root, file)

# Example usage:
            
all_files = []
startpath = '../full_texts/'
for filepath in list_files(startpath):
    if ".json" in filepath:
        all_files.append(filepath)
        

In [9]:
len(all_files)

15352

In [10]:

import re

def find_digital_twin_definitions(text):
    regex_patterns = [
        r'\b(?:digital twin(?:s)?)\s+(?:is\s+)?(?:defined\s+as)\s+.+?(?=\.\s|\n|$)', # good one "digital twin is deined as"
        r'\b(?:defined)\s+(?:digital twin(?:s)?)\s+(?:as)\s+.+?(?=\.\s|\n|$)', # good one "xxx defined digital twin as"
        r'\b(?:digital twin(?:s)?)\s+(?:is\s+)?(?:defined\s+as)\s+.+?(?=\.\s|\n|$)', #good one "repeated"
        r'\bdigital twin(?:s)? is a .+?(?=\.\s|\n|$)', # good one "digital twin is a ....."
        r'\b(?:digital twin(?:s)?)\s+(?:is\s+)?(?:described\s+as)\s+.+?(?=\.\s|\n|$)', #good one "digital twin/s is/are described as ....."
        r'\b(?:digital twin(?:s)?)\s+(?:is\s+)?(?:characterized\s+by)\s+.+?(?=\.\s|\n|$)', #few results "digital twin is characterized by ....."
        r'\b(?:digital twin(?:s)?)\s+(?:can\s+be\s+)?(?:defined\s+as)\s+.+?(?=\.\s|\n|$)', # Prompt: "digital twin can be defined as ..."
        r'\b(?:in\s+the\s+context\s+of\s+digital\s+twin(?:s)?)(?:,\s+)?(?:\w+\s+)?(?:it\s+refers\s+to)\s+.+?(?=\.\s|\n|$)', # Additional pattern for introducing "digital twin" without explicit "defined" or "described"
    ]

    definitions = []
    for pattern in regex_patterns:
        matches = re.findall(pattern, text, re.IGNORECASE | re.DOTALL)  # Ignore case and match newline characters
        definitions.extend(matches)

    return definitions


In [11]:
all_files[0]

'../full_texts/10.1533/9781782422433.2.309/10.1533_9781782422433.2.309.json'

In [12]:
all_defs = []
iter = 0
all_definitions = []
for index, file in enumerate(all_files):
    try:
        title = d["full-text-retrieval-response"]["coredata"]["dc:title"]
    except:
        title = "unkonwn"
    data ={
        "file": file,
        "title": title
    }
    file_defs = []
    try:
        with open( file, 'r') as f:
            d = f.read()
        definitions = find_digital_twin_definitions(d)
        if len(definitions) > 0 :
            for definition in definitions:
                if not definition in file_defs:
                    # print(iter, index,  definition)
                    file_defs.append(definition)
                    iter += 1
            data["defs"] = file_defs
            all_definitions.append(data)
            print(index,len(all_definitions),  end="\r")
    except Exception as e:
        pass

    with open("definitions_v2.json", 'w') as fw:
        fw.write(json.dumps(all_definitions))
    # if len(all_definitions) > 30:
    #     break
        

In [29]:
all_definitions

[{'file': './full_texts/10.1053/j.semnuclmed.2022.10.007/10.1053_j.semnuclmed.2022.10.007.json',
  'metadata': {'eid': '1-s2.0-S0016508505006396',
   'dc:description': None,
   'openArchiveArticle': 'false',
   'prism:coverDate': '2005-04-30',
   'openaccessUserLicense': None,
   'prism:aggregationType': 'Journal',
   'prism:url': 'https://api.elsevier.com/content/article/pii/S0016508505006396',
   'link': [{'@_fa': 'true',
     '@rel': 'self',
     '@href': 'https://api.elsevier.com/content/article/pii/S0016508505006396'},
    {'@_fa': 'true',
     '@rel': 'scidir',
     '@href': 'https://www.sciencedirect.com/science/article/pii/S0016508505006396'}],
   'dc:format': 'application/json',
   'openaccessType': None,
   'pii': 'S0016-5085(05)00639-6',
   'prism:volume': '128',
   'prism:publisher': 'Published by Elsevier Inc.',
   'dc:title': 'Program pages for DDW 2005',
   'prism:copyright': 'Copyright © 2005 Published by Elsevier Inc. All rights reserved.',
   'openaccess': '0',
   'pr

In [30]:
definitions_df = pd.DataFrame(all_definitions)
definitions_df

,file,title,defs
0,../full_texts/10.1053/j.semnuclmed.2022.10.007...,unkonwn,[digital twin is a digital model of the actual...
1,../full_texts/10.1016/j.dcan.2023.02.017/10.10...,unkonwn,[digital twin is a simulation model of the phy...
2,../full_texts/10.1016/j.micpro.2023.104794/10....,unkonwn,[Digital Twins is a transformation of technolo...
3,../full_texts/10.1016/j.apmt.2018.11.003/10.10...,unkonwn,[digital twin is a virtual replica of the hard...
4,../full_texts/10.1016/j.jmsy.2020.08.001/10.10...,unkonwn,[digital twin is a very significant and promis...
...,...,...,...
543,../full_texts/10.1016/j.cca.2019.05.021/10.101...,unkonwn,[digital twin is a virtual version of a system...
544,../full_texts/10.1016/j.ifacol.2022.04.220/10....,unkonwn,[Digital Twin is defined as a set of adaptive ...
545,../full_texts/10.1016/j.engappai.2023.106813/1...,unkonwn,[digital twin is a virtual replica of a physic...
546,../full_texts/10.1016/j.energy.2023.127317/10....,unkonwn,[defined digital twin as a representation of a...


In [25]:
all_metadata = []
def getMetadata(filename):
    try:
        with open (filename, 'r') as f:
            data = json.loads(f.read())
        all_metadata.append({"file": filename, **data['full-text-retrieval-response']['coredata']})
    except:
        pass
ind = 0
for file in list_files(startpath):
    ind+=1
    getMetadata(file)
    print(ind, "out of ", len(file), end="\r")

all_metadata_df = pd.DataFrame(all_metadata)
all_metadata_df

,file,eid,dc:description,openArchiveArticle,prism:coverDate,openaccessUserLicense,prism:aggregationType,prism:url,dc:creator,link,...,prism:startingPage,dc:identifier,openaccessSponsorName,prism:issn,prism:volume,prism:issueIdentifier,prism:number,prism:issueName,articleNumber,prism:edition
0,../full_texts/10.1533/9781782422433.2.309/10.1...,3-s2.0-B9781782422426500119,This chapter introduces the most common sensor...,false,2014-12-31,None,EBook,https://api.elsevier.com/content/article/pii/B...,"[{'@_fa': 'true', '$': 'Hoult, N.A.'}, {'@_fa'...","[{'@_fa': 'true', '@rel': 'self', '@href': 'ht...",...,309,doi:10.1533/9781782422433.2.309,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,../full_texts/10.1533/9781780634425.83/10.1533...,3-s2.0-B9781843347613500048,Shanzhai culture in China has permeated everyd...,false,2014-12-31,None,EBook,https://api.elsevier.com/content/article/pii/B...,"{'@_fa': 'true', '$': 'Tam, Karen'}","[{'@_fa': 'true', '@rel': 'self', '@href': 'ht...",...,83,doi:10.1533/9781780634425.83,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,../full_texts/10.1533/9781845696825.1.114/10.1...,3-s2.0-B9781845693923500049,This chapter provides an introduction to the d...,false,2009-12-31,None,EBook,https://api.elsevier.com/content/article/pii/B...,"[{'@_fa': 'true', '$': 'Shinozuka, M'}, {'@_fa...","[{'@_fa': 'true', '@rel': 'self', '@href': 'ht...",...,113,doi:10.1533/9781845696825.1.114,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,../full_texts/10.1533/9781845695224.index/10.1...,3-s2.0-B9781845693251500201,Unknown,false,2008-12-31,None,EBook,https://api.elsevier.com/content/article/pii/B...,NaN,"[{'@_fa': 'true', '@rel': 'self', '@href': 'ht...",...,334,doi:10.1533/9781845695224.index,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,../full_texts/10.1533/9781845694463.1.94/10.15...,3-s2.0-B9781845692957500058,None,False,2008-12-31,None,EBook,https://api.elsevier.com/content/article/pii/B...,"[{'@_fa': 'true', '$': 'Istook, C.L.'}]",[{'@href': 'https://api.elsevier.com/content/a...,...,94,doi:10.1533/9781845694463.1.94,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15347,../full_texts/10.1182/blood-2009-10-251413/10....,1-s2.0-S0006497120350965,Abstract Chimeric fusion genes are highly prev...,true,2010-04-29,http://www.elsevier.com/open-access/userlicens...,Journal,https://api.elsevier.com/content/article/pii/S...,"[{'@_fa': 'true', '$': 'Bateman, Caroline M.'}...","[{'@_fa': 'true', '@rel': 'self', '@href': 'ht...",...,3553,doi:10.1182/blood-2009-10-251413,None,00064971,115,17,17,NaN,NaN,NaN
15348,../full_texts/10.1182/blood-2011-07-366542/10....,1-s2.0-S0006497120403337,Abstract The timing and developmental sequence...,true,2011-11-17,http://www.elsevier.com/open-access/userlicens...,Journal,https://api.elsevier.com/content/article/pii/S...,"[{'@_fa': 'true', '$': 'Cazzaniga, Giovanni'},...","[{'@_fa': 'true', '@rel': 'self', '@href': 'ht...",...,5559,doi:10.1182/blood-2011-07-366542,None,00064971,118,20,20,NaN,NaN,NaN
15349,../full_texts/10.1182/bloodadvances.2021005703...,1-s2.0-S2473952922000106,Abstract Genetic analysis of leukemic clones i...,false,2022-04-12,http://creativecommons.org/licenses/by-nc-nd/4.0/,Journal,https://api.elsevier.com/content/article/pii/S...,"[{'@_fa': 'true', '$': 'Bang, Benedicte'}, {'@...","[{'@_fa': 'true', '@rel': 'self', '@href': 'ht...",...,2275,doi:10.1182/bloodadvances.2021005703,None,24739529,6,7,7,NaN,NaN,NaN
15350,../full_texts/10.1182/blood-2005-09-3902/10.11...,1-s2.0-S0006497120524476,Abstract Immune mechanisms are involved in the...,true,2006-10-15,http://www.elsevier.com/open-access/userlicens...,Journal,https://api.elsevier.com/content/article/pii/S...,"[{'@_fa': 'true', '$': 'Wlodarski, Marcin W.'}...","[{'@_fa': 'true', '@rel': 'self', '@href': 'ht...",...,2632,doi:10.1182/blood-2005-09-3902,None,00064971,108,8,8,NaN,NaN,NaN


In [31]:
definitions_df = definitions_df.explode(column="defs").reset_index(drop=True)


,file,title,defs
0,../full_texts/10.1053/j.semnuclmed.2022.10.007...,unkonwn,digital twin is a digital model of the actual ...
1,../full_texts/10.1016/j.dcan.2023.02.017/10.10...,unkonwn,digital twin is a simulation model of the phys...
2,../full_texts/10.1016/j.micpro.2023.104794/10....,unkonwn,Digital Twins is a transformation of technolog...
3,../full_texts/10.1016/j.micpro.2023.104794/10....,unkonwn,Digital Twins is a collaboration of various in...
4,../full_texts/10.1016/j.apmt.2018.11.003/10.10...,unkonwn,digital twin is a virtual replica of the hardw...
...,...,...,...
842,../full_texts/10.1016/j.ifacol.2022.04.220/10....,unkonwn,Digital Twin is defined as a set of adaptive m...
843,../full_texts/10.1016/j.ifacol.2022.04.220/10....,unkonwn,Digital Twin is a Digital Twin devoted to the ...
844,../full_texts/10.1016/j.engappai.2023.106813/1...,unkonwn,digital twin is a virtual replica of a physica...
845,../full_texts/10.1016/j.energy.2023.127317/10....,unkonwn,defined digital twin as a representation of a ...


In [33]:
definitions_df_metadata = pd.merge(definitions_df, all_metadata_df, on="file")
definitions_df_metadata

,file,title,defs,eid,dc:description,openArchiveArticle,prism:coverDate,openaccessUserLicense,prism:aggregationType,prism:url,...,prism:startingPage,dc:identifier,openaccessSponsorName,prism:issn,prism:volume,prism:issueIdentifier,prism:number,prism:issueName,articleNumber,prism:edition
0,../full_texts/10.1053/j.semnuclmed.2022.10.007...,unkonwn,digital twin is a digital model of the actual ...,1-s2.0-S0001299822000940,"Developments in artificial intelligence, parti...",false,2023-05-31,None,Journal,https://api.elsevier.com/content/article/pii/S...,...,457,doi:10.1053/j.semnuclmed.2022.10.007,None,00012998,53,3,3,Hematology,NaN,NaN
1,../full_texts/10.1016/j.dcan.2023.02.017/10.10...,unkonwn,digital twin is a simulation model of the phys...,1-s2.0-S2352864823000524,The Metaverse depicts a parallel digitalized w...,false,2023-03-03,http://creativecommons.org/licenses/by-nc-nd/4.0/,Journal,https://api.elsevier.com/content/article/pii/S...,...,NaN,doi:10.1016/j.dcan.2023.02.017,Chongqing University of Posts and Telecommunio...,23528648,NaN,NaN,NaN,NaN,NaN,NaN
2,../full_texts/10.1016/j.micpro.2023.104794/10....,unkonwn,Digital Twins is a transformation of technolog...,1-s2.0-S0141933123000406,The Digital twins will duplicate the actual ob...,false,2023-04-30,None,Journal,https://api.elsevier.com/content/article/pii/S...,...,104794,doi:10.1016/j.micpro.2023.104794,None,01419331,98,NaN,NaN,NaN,104794,NaN
3,../full_texts/10.1016/j.micpro.2023.104794/10....,unkonwn,Digital Twins is a collaboration of various in...,1-s2.0-S0141933123000406,The Digital twins will duplicate the actual ob...,false,2023-04-30,None,Journal,https://api.elsevier.com/content/article/pii/S...,...,104794,doi:10.1016/j.micpro.2023.104794,None,01419331,98,NaN,NaN,NaN,104794,NaN
4,../full_texts/10.1016/j.apmt.2018.11.003/10.10...,unkonwn,digital twin is a virtual replica of the hardw...,1-s2.0-S2352940718304931,The customized production of complex component...,false,2019-03-31,None,Journal,https://api.elsevier.com/content/article/pii/S...,...,59,doi:10.1016/j.apmt.2018.11.003,None,23529407,14,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
842,../full_texts/10.1016/j.ifacol.2022.04.220/10....,unkonwn,Digital Twin is defined as a set of adaptive m...,1-s2.0-S240589632200221X,The development of future Industrial Cyber-Phy...,false,2022-12-31,None,Journal,https://api.elsevier.com/content/article/pii/S...,...,360,doi:10.1016/j.ifacol.2022.04.220,None,24058963,55,2,2,14th IFAC Workshop on Intelligent Manufacturin...,NaN,NaN
843,../full_texts/10.1016/j.ifacol.2022.04.220/10....,unkonwn,Digital Twin is a Digital Twin devoted to the ...,1-s2.0-S240589632200221X,The development of future Industrial Cyber-Phy...,false,2022-12-31,None,Journal,https://api.elsevier.com/content/article/pii/S...,...,360,doi:10.1016/j.ifacol.2022.04.220,None,24058963,55,2,2,14th IFAC Workshop on Intelligent Manufacturin...,NaN,NaN
844,../full_texts/10.1016/j.engappai.2023.106813/1...,unkonwn,digital twin is a virtual replica of a physica...,1-s2.0-S0952197623009971,This narrative review paper explores the diver...,false,2023-11-30,None,Journal,https://api.elsevier.com/content/article/pii/S...,...,106813,doi:10.1016/j.engappai.2023.106813,None,09521976,126,NaN,NaN,NaN,106813,NaN
845,../full_texts/10.1016/j.energy.2023.127317/10....,unkonwn,defined digital twin as a representation of a ...,1-s2.0-S0360544223007119,The gradual energy consumption increments and ...,false,2023-07-01,None,Journal,https://api.elsevier.com/content/article/pii/S...,...,127317,doi:10.1016/j.energy.2023.127317,None,03605442,274,NaN,NaN,NaN,127317,NaN


In [41]:
def sub(x):
    try:
        return x["$"]
    except:
        return None
definitions_df_metadata_subjects = definitions_df_metadata[['file', 'dcterms:subject', 'dc:title']].explode('dcterms:subject')
definitions_df_metadata_subjects["subject"] = definitions_df_metadata_subjects['dcterms:subject'].apply(lambda x :  sub(x))

In [58]:
vc = definitions_df_metadata_subjects["subject"].value_counts()[2:]

subjects_df = pd.DataFrame(vc)
subjects_df[subjects_df["count"]>1]

,count
subject,
Industry 4.0,123
Simulation,40
Internet of Things,37
digital twin,32
Machine learning,32
...,...
Applicability of the transportation system,2
Smart Factory,2
Uncertain event,2


In [ ]:
repeated = {}
unique = []
for i , definition in enumerate(all_definitions):
    if len(definition["defs"]) >0:
        print(len(definition["defs"]), definition["title"])
        for d in definition["defs"]:
            if not d.lower() in unique:
                unique.append(d)
            elif d.lower() in repeated.keys():
                repeated[d.lower()] += 1
            else:
                repeated[d.lower()] = 1
            print(d)
        print(" \n")
    

In [68]:
repeated

{'digital twin is a digital (virtual) copy of a physical (real) system or process created to optimize system performance': 1,
 'digital twin is a virtual model of a process, product or service': 1,
 'digital twin is a digital replica of a living or non-living physical entity': 3,
 'digital twin is a virtual replica that mimics the behaviour of an actual object or process throughout its existence': 1,
 'digital twin is a virtual instance of a physical system (twin) that is continually updated with the latter’s performance, maintenance, and health status data throughout the physical system’s life cycle': 1,
 'digital twin is a virtual representation of real-world entities and processes, synchronized at a specified frequency and fidelity': 1,
 'digital twin is a set of virtual information constructs that fully describes a potential or actual physical manufactured product from the micro atomic level to the macro geometrical level': 2,
 'digital twin is a systematic model with rank going fo

In [69]:
len(unique)

808